# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [198]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import pickle

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, ne_chunk, pos_tag, sent_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score, confusion_matrix, precision_score, recall_score, make_scorer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import xgboost as xgb
from xgboost import XGBClassifier

[nltk_data] Downloading package punkt to /home/wefo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/wefo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/wefo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/wefo/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/wefo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /home/wefo/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [25]:
# load data from database
engine = create_engine('sqlite:///sql_database.db')
df = pd.read_sql("SELECT * FROM cleaned_data", engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [113]:
stop_words = stopwords.words('english')

def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [177]:
pipeline = Pipeline([
    ('tokenizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [65]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tokenizer',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9bae973c20>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        max_depth=5)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [66]:
preds = pipeline.predict(X_test)

In [207]:
pred = pd.DataFrame(preds, columns=y_train.columns, index=X_test.index)

In [237]:
def classiffier_report(y_true, y_pred, average='weighted'):
    results = pd.DataFrame(index=y_true.columns, columns=['precision','recall','f1-score'])
    results_fmt = "{:<22} Precision: {:.3f}   Recall: {:.3f}   F1-score: {:.3f}"
    for col in y_test.columns:
        res = [ col.upper(),
               precision_score(y_true[col], y_pred[col], average=average),
               recall_score(y_true[col], y_pred[col], average=average),
               f1_score(y_true[col], y_pred[col], average=average)]
        print(results_fmt.format(*res))

In [239]:
classiffier_report(y_test, rf_preds)

RELATED                Precision: 0.785   Recall: 0.659   F1-score: 0.684
REQUEST                Precision: 0.875   Recall: 0.864   F1-score: 0.868
OFFER                  Precision: 0.990   Recall: 0.986   F1-score: 0.988
AID_RELATED            Precision: 0.749   Recall: 0.751   F1-score: 0.749
MEDICAL_HELP           Precision: 0.909   Recall: 0.889   F1-score: 0.897
MEDICAL_PRODUCTS       Precision: 0.941   Recall: 0.904   F1-score: 0.919
SEARCH_AND_RESCUE      Precision: 0.963   Recall: 0.927   F1-score: 0.943
SECURITY               Precision: 0.970   Recall: 0.966   F1-score: 0.968
MILITARY               Precision: 0.965   Recall: 0.940   F1-score: 0.951
CHILD_ALONE            Precision: 1.000   Recall: 1.000   F1-score: 1.000
WATER                  Precision: 0.958   Recall: 0.938   F1-score: 0.945
FOOD                   Precision: 0.924   Recall: 0.910   F1-score: 0.915
SHELTER                Precision: 0.927   Recall: 0.906   F1-score: 0.914
CLOTHING               Precision: 0.98

### 6. Improve your model
Use grid search to find better parameters. 

In [199]:
def custom_f1_score(y_true, y_pred):
    if isinstance(y_true, pd.DataFrame):
        y_true = y_true.values
    diffs = []
    for col in range(y_true.shape[1]):
        err = f1_score(y_true[:,col], y_pred[:,col], average='weighted')
        diffs.append(err)
    return np.mean(diffs)
    


parameters = [
    {
        'clf__estimator': [RandomForestClassifier()],
        'clf__estimator__n_estimators': [10, 20],
        'clf__estimator__max_depth': [1, 5],
        'clf__estimator__class_weight': [None, 'balanced']
    },
    {
        'clf__estimator': [AdaBoostClassifier()],
        'clf__estimator__n_estimators': [10, 20],
    }

]

scoring = make_scorer(custom_f1_score, greater_is_better=True)

cv = GridSearchCV(pipeline, parameters, scoring=scoring)

cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('tokenizer',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f9b8209ad40>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid=[{'clf__estimator': [RandomForestClassifier()],
                          'clf__estimator__class_weight': [None, 'balanced'],
                          'clf__estimator__max_depth': [1, 5],
                          'clf__estimator__n_estimators': [10, 20]},
                         {'clf__estimator': [AdaBoostClassifier(n_estimators=20)],
                          'clf__estimator__n_estimators': [10, 20]}],
             scoring=make_scorer(custom_f1_score))

In [200]:
cv.best_params_

{'clf__estimator': AdaBoostClassifier(n_estimators=20),
 'clf__estimator__n_estimators': 20}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [201]:
y_pred = cv.predict(X_test)

In [202]:
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns, index=X_test.index)

In [241]:
first_model_results

In [242]:
classiffier_report(y_test, y_pred_df)

RELATED                Precision: 0.772   Recall: 0.764   F1-score: 0.675
REQUEST                Precision: 0.874   Recall: 0.883   F1-score: 0.870
OFFER                  Precision: 0.990   Recall: 0.994   F1-score: 0.992
AID_RELATED            Precision: 0.738   Recall: 0.737   F1-score: 0.729
MEDICAL_HELP           Precision: 0.904   Recall: 0.922   F1-score: 0.906
MEDICAL_PRODUCTS       Precision: 0.948   Recall: 0.956   F1-score: 0.948
SEARCH_AND_RESCUE      Precision: 0.970   Recall: 0.977   F1-score: 0.970
SECURITY               Precision: 0.967   Recall: 0.980   F1-score: 0.973
MILITARY               Precision: 0.964   Recall: 0.971   F1-score: 0.966
CHILD_ALONE            Precision: 1.000   Recall: 1.000   F1-score: 1.000
WATER                  Precision: 0.962   Recall: 0.965   F1-score: 0.963
FOOD                   Precision: 0.933   Recall: 0.937   F1-score: 0.930
SHELTER                Precision: 0.939   Recall: 0.945   F1-score: 0.940
CLOTHING               Precision: 0.98

### 9. Export your model as a pickle file

In [185]:
# Save to file in the current working directory
pkl_filename = "final_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.